In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('Datasets/syntheticDataset_O50.csv') # 데이터 불러오기

df['hour'] = df['request_time'] // 3600  # 초 단위 -> 시간 단위 버킷

object_ids = df['object_ID'].unique()
object_ids.sort()
num_objects = len(object_ids)
print("총 객체 수: ", num_objects)

pivot = df.groupby(['hour', 'object_ID']).size().unstack(fill_value=0)
pivot = pivot.reindex(columns=object_ids, fill_value=0) # 객체 ID 순서 맞추기

# 확률 벡터로 정규화
probs = pivot.div(pivot.sum(axis=1), axis=0).fillna(0)

m, K = 20, 10
X = []
y = []

for i in range(len(probs) - m - K):
  x_seq = probs.iloc[i:i+m].values  # shape: (m, d)
  x_next = probs.iloc[i+m:i+m+K].values
  X.append(x_seq)
  y.append(x_next)

X = np.array(X)
y = np.array(y)

print("X.shape =", X.shape)
print("y.shape =", y.shape)

총 객체 수:  50
X.shape = (55, 20, 50)
y.shape = (55, 10, 50)


In [ ]:
# LSTM 예측 모델
# 과거 일정 시간 동안의 요청 분포를 받아서 앞으로의 일정 시간 동안의 요청 분포를 예측하는 구조
# Encoder: 과거 m시간 동안의 요청 분포((m, 1033) 시퀀스)를 받아서, 마지막 hidden state와 cell state로 응축된 정보를 생성 -> context vector로 요약
# Decoder: Encoder에서 받은 context vector를 바탕으로, 앞으로 k시간 동안 어떤 객체들이 얼마나 요청될지를 시퀀스 형태로 한 시간씩 예측
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, TimeDistributed, RepeatVector

def build_seq2seq_model(m, K, num_objects):
  # ----- Encoder -----
  encoder_inputs = Input(shape=(m, num_objects)) # (batch, m(시간), num_objects(객체수))
  encoder_lstm = LSTM(128, return_state=True) # LSTM이 마지막 시점의 hidden state와 cell state를 반환
  encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs) # state_h와 state_c는 Decoder의 초기 상태로 사용됨
  encoder_states = [state_h, state_c] # Encoder의 상태를 Decoder에 전달하기 위해 리스트로 묶음

  # ----- Decoder -----
  decoder_inputs = RepeatVector(K)(encoder_outputs) # (batch, K, 128): Encoder의 마지막 출력을 K번 복제해서 Decoder 입력으로 사용, Decoder는 K시간 동안 예측을 수행
  decoder_lstm = LSTM(128, return_sequences=True) # LSTM의 hidden size는 64, return_sequences=True는 각 시점마다 출력을 반환
  decoder_outputs = decoder_lstm(decoder_inputs, initial_state=encoder_states)

  # 객체별 확률 예측 (각 시점당 num_objects 개 출력)
  decoder_dense = TimeDistributed(Dense(num_objects, activation='softmax')) # softmax는 확률 분포로 만들어줌
  output_seq = decoder_dense(decoder_outputs)

  # ----- 모델 구성 -----
  model = Model(encoder_inputs, output_seq)
  model.compile(optimizer='adam', loss='mse', metrics=['mae'])
  return model


num_objects = 50  # 객체 수
m = 20              # 과거 시간: 입력 시퀀스 길이
K = 10             # 미래 시간: 출력 시퀀스 길이

model = build_seq2seq_model(m, K, num_objects)

# 데이터: X shape = (samples, 20, 50), y shape = (samples, 10, 50)
history = model.fit(X, y, epochs=30, batch_size=32, validation_split=0.1)

# 학습 후 추론 코드
from sklearn.model_selection import train_test_split

# 시퀀스 데이터를 학습/검증/테스트로 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 학습 시에도 X_train, y_train 사용
history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.1)

# 추론 (예측)
y_pred = model.predict(X_test)
print("예측 결과 형태:", y_pred.shape)

Epoch 1/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 821ms/step - loss: 8.4627e-04 - mae: 0.0141 - val_loss: 0.0013 - val_mae: 0.0130
Epoch 2/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 137ms/step - loss: 8.2983e-04 - mae: 0.0140 - val_loss: 0.0013 - val_mae: 0.0130
Epoch 3/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - loss: 8.5760e-04 - mae: 0.0142 - val_loss: 0.0013 - val_mae: 0.0130
Epoch 4/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - loss: 8.8486e-04 - mae: 0.0144 - val_loss: 0.0013 - val_mae: 0.0129
Epoch 5/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - loss: 8.6540e-04 - mae: 0.0143 - val_loss: 0.0013 - val_mae: 0.0129
Epoch 6/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - loss: 8.6730e-04 - mae: 0.0143 - val_loss: 0.0013 - val_mae: 0.0129
Epoch 7/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - loss: 8.2528e-04 - mae: 0.0140 - val_loss: 0.0013 - val_mae: 0.0128
Epoch 8/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - loss: 8.3814e-04 - mae: 0.0141 - val_loss: 0.0013 - val_mae: 0.0128
Epoch 9/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 100

In [ ]:
# 캐시 정책 구현
# 예측 결과를 기반으로 DeepCache의 캐싱 정책을 구현하고 이를 통해 기본 캐싱 전략과 비교평가
M = 5 # 캐시에 넣을 상위 M개 객체
top_objects_each_t = []

# 예측 결과에서 Top-M 객체 추출
for i in range(len(y_pred)):
    for t in range(K - 1):    # t+1 예측을 위해 K-1까지만
        next_probs = y_pred[i, t+1]  # 다음 시점 확률 분포
        top_indices = next_probs.argsort()[-M:][::-1]  # 상위 M개 객체 인덱스
        top_objects_each_t.append(top_indices)

# 실제 요청 로그 준비
actual_requests = df.sort_values('request_time')['object_ID'].tolist()

In [ ]:
# 캐시 시뮬레이션 (LRU + 예측 기반) -> 성능 평가
from collections import deque

CACHE_SIZE = 20
cache = deque(maxlen=CACHE_SIZE)
hit = 0
total = 0
fake_insert_idx = 0

for t, req in enumerate(actual_requests):
  # 예측한 객체를 먼저 캐시에 넣기
  if t % 100 == 0 and fake_insert_idx < len(top_objects_each_t):
    fake_objs = top_objects_each_t[fake_insert_idx]
    for obj in fake_objs:
      if obj not in cache:
        cache.append(obj)
    fake_insert_idx += 1

  total += 1
  if req in cache:
    hit += 1
  else:
    cache.append(req)

print(f"DeepCache 기반 캐시 hit ratio: {hit / total:.4f}")

DeepCache 기반 캐시 hit ratio: 0.6571


In [ ]:
# 기본 LRU 캐시와 성능 비교